# Markov Chains and Chord Progressions

In [2]:
import numpy as np
from music21 import *

In [3]:
# only run the below to configure music21 for viewing/generating sheet music
# configure.run()

In [4]:
# set up chords
chord_numbers = np.array(range(0,7))
print("Chord numbers:", chord_numbers)

chord_names = np.array(["major_first", "minor_second", "minor_third", "major_fourth", "major_fifth", "minor_sixth", "dim_seventh"])
print("Chord names:", chord_names)

chord_combined = np.column_stack((chord_names, chord_numbers))
print("Represented as a combined array:", chord_combined)

Chord numbers: [0 1 2 3 4 5 6]
Chord names: ['major_first' 'minor_second' 'minor_third' 'major_fourth' 'major_fifth'
 'minor_sixth' 'dim_seventh']
Represented as a combined array: [['major_first' '0']
 ['minor_second' '1']
 ['minor_third' '2']
 ['major_fourth' '3']
 ['major_fifth' '4']
 ['minor_sixth' '5']
 ['dim_seventh' '6']]


In [5]:
# transition diagram - not normalized 

chord_transitions = np.array([
                            [0, 1, 1, 1, 1, 1, 1], # major first
                            [0, 0, 0, 0, 0, 1, 1], # minor 2nd
                            [0, 1, 0, 1, 0, 1, 0], # minor third
                            [1, 1, 0, 0, 1, 0, 1], # major fourth
                            [1, 0, 0, 0, 0, 0, 0], # major fifth
                            [0, 1, 0, 1, 1, 0, 0], # minor 6
                            [1, 0, 0, 0, 0, 0, 0], # dim 7
    
    
                            ])
chord_transitions

array([[0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0]])

In [6]:
# normalize chord matrix

# convert to transition matrix 

# first setup matrix with zeros
n = chord_transitions.shape[0]
chord_matrix = np.zeros_like(chord_transitions, dtype=float)

# normalize values to make rows sum to 1
for i in range(n):
    if chord_transitions[i].sum() == 0:
        chord_matrix[i] = np.ones(n) / n
    else:
        chord_matrix[i] = chord_transitions[i] / chord_transitions[i].sum()

chord_matrix

array([[0.        , 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.16666667],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5       , 0.5       ],
       [0.        , 0.33333333, 0.        , 0.33333333, 0.        ,
        0.33333333, 0.        ],
       [0.25      , 0.25      , 0.        , 0.        , 0.25      ,
        0.        , 0.25      ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.33333333, 0.        , 0.33333333, 0.33333333,
        0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [7]:
# initial distribution
initial_dist = np.array([1, 0, 0, 0, 0, 0, 0])  # always start on 1

In [8]:
# simulation

min_steps = 32 # minimum number of steps - progression will need to end on 1
max_steps = 40 # maximum number of steps - progression will need to end on 1 

initial_chord = 0   # always start on 1 - index at 0
chord_progression = [initial_chord] # set up list to hold chord progession
for s in range(max_steps):
    prev_chord = chord_progression[-1]
    next_chord = np.random.choice(chord_numbers, p = chord_matrix[prev_chord])
    chord_progression.append(next_chord)

    if s >= min_steps:
        if next_chord == 0:
            break


print("The final chord progression is ", len(chord_progression), " chords long and has the following progression: ", chord_progression)


The final chord progression is  34  chords long and has the following progression:  [0, 4, 0, 3, 4, 0, 5, 3, 1, 6, 0, 1, 6, 0, 6, 0, 5, 1, 6, 0, 6, 0, 3, 6, 0, 1, 6, 0, 2, 3, 6, 0, 4, 0]


In [18]:
# construct notes for use in music21 chord constructor
notes = ["C", "D", "E", "F", "G", "A", "B", "C", "D", "E", "F"]
octaves = ["4", "4", "4", "4", "4", "4", "4", "5","5","5","5"]
note_oct = []


In [10]:
# example of note octave notation 
C_4 = str(notes[0] + octaves[0])
C_4

'C4'

In [19]:
# pair notes with octaves

for i in range(0, len(notes)):
    n = str(notes[i] + octaves[i])
    note_oct.append(n)
note_oct

['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5', 'D5', 'E5', 'F5']

In [12]:
chord

<module 'music21.chord' from 'c:\\Users\\Myranda\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\music21\\chord\\__init__.py'>

In [20]:
# construct chords using music21
chord_list = []

for c in chord_progression:
    starting_note = note_oct[c]
    third = note_oct[c+2]
    fifth = note_oct[c+4]
    ch = chord.Chord([starting_note, third, fifth])
    chord_list.append(ch)

print(chord_list)

[<music21.chord.Chord C4 E4 G4>, <music21.chord.Chord G4 B4 D5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord F4 A4 C5>, <music21.chord.Chord G4 B4 D5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord A4 C5 E5>, <music21.chord.Chord F4 A4 C5>, <music21.chord.Chord D4 F4 A4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord D4 F4 A4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord A4 C5 E5>, <music21.chord.Chord D4 F4 A4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord F4 A4 C5>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord D4 F4 A4>, <music21.chord.Chord B4 D5 F5>, <music21.chord.Chord C4 E4 G4>, <music21.chord.Chord E4 G4 B4>, <music21.chord.Chord F4 A4 C5>, <music21.chord.Chord B4 D5 F5>, <music2

In [14]:
#configure.run()

In [15]:
chord_list[0].show("musicxml")

In [21]:
# add all chords to a stream

chord_stream = stream.Stream()
for c in chord_list:
    chord_stream.append(c)

chord_stream.show("text")

{0.0} <music21.chord.Chord C4 E4 G4>
{1.0} <music21.chord.Chord G4 B4 D5>
{2.0} <music21.chord.Chord C4 E4 G4>
{3.0} <music21.chord.Chord F4 A4 C5>
{4.0} <music21.chord.Chord G4 B4 D5>
{5.0} <music21.chord.Chord C4 E4 G4>
{6.0} <music21.chord.Chord A4 C5 E5>
{7.0} <music21.chord.Chord F4 A4 C5>
{8.0} <music21.chord.Chord D4 F4 A4>
{9.0} <music21.chord.Chord B4 D5 F5>
{10.0} <music21.chord.Chord C4 E4 G4>
{11.0} <music21.chord.Chord D4 F4 A4>
{12.0} <music21.chord.Chord B4 D5 F5>
{13.0} <music21.chord.Chord C4 E4 G4>
{14.0} <music21.chord.Chord B4 D5 F5>
{15.0} <music21.chord.Chord C4 E4 G4>
{16.0} <music21.chord.Chord A4 C5 E5>
{17.0} <music21.chord.Chord D4 F4 A4>
{18.0} <music21.chord.Chord B4 D5 F5>
{19.0} <music21.chord.Chord C4 E4 G4>
{20.0} <music21.chord.Chord B4 D5 F5>
{21.0} <music21.chord.Chord C4 E4 G4>
{22.0} <music21.chord.Chord F4 A4 C5>
{23.0} <music21.chord.Chord B4 D5 F5>
{24.0} <music21.chord.Chord C4 E4 G4>
{25.0} <music21.chord.Chord D4 F4 A4>
{26.0} <music21.chord.

In [ ]:
chord_stream.show("musicxml")

Now we have a chord progression! Let's add a melody line.

In [ ]:
note_transitions = ["up", "down", "same"]


In [27]:
# playground

first_note = "C4"
first_chord = chord_list[0]

next_chord = chord_list[1]
next_note_index = np.random.choice([0,1,2], p = [1/3,1/3,1/3])
next_note = next_chord.notes[next_note_index]
next_note

<music21.note.Note D>

In [47]:
# go through a melody line (following some music rules: tritones, major sevenths)

# set first note as tonic
melody = [chord_list[0].notes[0]]
melody_stream = stream.Stream()
melody_stream.append(chord_list[0].notes[0])

# find next note, set to same octave
for i in range(1, len(chord_list)-1):
    next_chord = chord_list[i]
    next_note_idx = np.random.choice([0,1,2], p = [1/3,1/3,1/3])
    next_note = next_chord.notes[next_note_idx]
    next_note.octave = 4

    # check for tritones (augmented 4ths)
    prev_note = melody[-1]
    note_interval = interval.Interval(prev_note, next_note)
    if note_interval.niceName == "Augmented Fourth":
        possible_notes = [0,1,2]
        possible_notes.remove(next_note_idx)
        next_note_idx = np.random.choice(possible_notes, p = [0.5,0.5])
        next_note = next_chord.notes[next_note_idx]
        next_note.octave = 4
    
    # TODO: check for large intervals

    # add to melody line
    melody.append(next_note)
    melody_stream.append(next_note)

# set last note as tonic
next_note = chord_list[-1].notes[0]
melody.append(next_note)
melody_stream.append(next_note)

melody_stream.show("text")

{0.0} <music21.note.Note C>
{1.0} <music21.note.Note G>
{2.0} <music21.note.Note C>
{3.0} <music21.note.Note F>
{4.0} <music21.note.Note D>
{5.0} <music21.note.Note C>
{6.0} <music21.note.Note A>
{7.0} <music21.note.Note C>
{8.0} <music21.note.Note A>
{9.0} <music21.note.Note D>
{10.0} <music21.note.Note E>
{11.0} <music21.note.Note D>
{12.0} <music21.note.Note B>
{13.0} <music21.note.Note C>
{14.0} <music21.note.Note F>
{15.0} <music21.note.Note E>
{16.0} <music21.note.Note E>
{17.0} <music21.note.Note A>
{18.0} <music21.note.Note D>
{19.0} <music21.note.Note C>
{20.0} <music21.note.Note F>
{21.0} <music21.note.Note G>
{22.0} <music21.note.Note F>
{23.0} <music21.note.Note F>
{24.0} <music21.note.Note C>
{25.0} <music21.note.Note D>
{26.0} <music21.note.Note F>
{27.0} <music21.note.Note E>
{28.0} <music21.note.Note G>
{29.0} <music21.note.Note F>
{30.0} <music21.note.Note F>
{31.0} <music21.note.Note C>
{32.0} <music21.note.Note B>
{33.0} <music21.note.Note C>


In [53]:
# generate melody thru intervals
# unison, up (second, third, fourth, fifth, sixth, seventh), down (etc)
# melody line go up or down
# if went up by fourth or greater, must then go down, and vice versa

# rows (for each interval): up small, up large, unison, down small, down large
# columns: unison, up (interval), down(interval)

# occurence for each given past interval is up small 
# unison, up(all), down(all)
up_down_small = [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0] # also applies to unison
up_large = [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
down_large = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

num_states = 13 # unison, plus up and down all intervals, no octave jumps

transition_states = np.zeros((num_states, num_states))
transition_states[0] = up_down_small
for i in range(1,3):
    transition_states[i] = up_down_small

for i in range(3, 7):
    transition_states[i] = up_large

for i in range(7, 9):
    transition_states[i] = up_down_small

for i in range(9, num_states):
    transition_states[i] = down_large

transition_states

array([[1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [48]:
melody_stream.show("musicxml")

In [33]:
# playground
# can set octaves
melody[0].octave = 5
melody[0].octave

5

In [42]:
f_note = note.Note("F5")
b_note = note.Note("B5")

interval.Interval(f_note, b_note).niceName

'Augmented Fourth'

In [46]:
a = [1,2,3]
a.remove(1)
a

[2, 3]